## Training neural networks

This notebook describes the workflow for training neural networks

In [1]:
%load_ext autoreload
%autoreload 2

In [25]:
from pathlib import Path
from functools import reduce

import yaml
import numpy as np
import keras
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
from dstools.reproducibility import make_filename

from yass import util

In [3]:
# for reference
print('YASS version is: {}'.format(util.get_version()))

YASS version is: fe1bd86 updates detect.run to use new separated code


In [4]:
path_to_data = Path('~', 'data').expanduser()
path_to_models = Path(path_to_data, 'models')

## Loading training data

In [5]:
path_to_sets = Path(path_to_data, 'training-sets')

path_to_x = Path(path_to_sets, 'x-triage-14-Aug-2018@21-28-23.npy')
path_to_y = Path(path_to_sets, 'y-triage-14-Aug-2018@21-28-23.npy')

x_triage = np.load(path_to_x)
y_triage = np.load(path_to_y)

In [6]:
def make_model(x_train, input_shape):
    n_data, window_size, n_channels, _ = x_train.shape

    model = Sequential()
        
#     model.add(MaxPooling2D(pool_size=(2, 1), data_format="channels_last", padding='same'))
    
#     model.add(Dropout(0.75))

    model.add(Conv2D(10, kernel_size=(5, 5),
                     padding='same', activation='relu', use_bias=True,
                     data_format="channels_last", input_shape=input_shape))


#     model.add(Conv2D(70, kernel_size=(window_size, 1),
#                      padding='valid', activation='relu', use_bias=True,
#                      data_format="channels_last"))
    
    model.add(Conv2D(10, kernel_size=(5, 5),
                     padding='same', activation='relu', use_bias=True,
                     data_format="channels_last"))


#     model.add(Conv2D(70, kernel_size=(1, n_channels),
#                      padding='valid', activation='relu', use_bias=True,
#                      data_format="channels_last"))
    
    model.add(Conv2D(10, kernel_size=(5, 5),
                     padding='same', activation='linear', use_bias=True,
                     data_format="channels_last"))
        
#     model.add(MaxPooling2D(pool_size=(1, 2), data_format="channels_last", padding='same'))
#     model.add(Dropout(0.75))
    
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    
    model.summary()
    
    # initiate RMSprop optimizer
    opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
    opt = keras.optimizers.adam(lr=0.001)

    model.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    
    return model


# Triage Ttaining

In [7]:
# import models
x = x_triage[:, : , :, np.newaxis]

(x_train, x_test,
y_train, y_test) = train_test_split(x, y_triage, test_size=0.3)

_, wf, ch, _ = x_train.shape

m = make_model(x, (wf, ch, 1))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 41, 7, 10)         260       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 41, 7, 10)         2510      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 41, 7, 10)         2510      
_________________________________________________________________
flatten_1 (Flatten)          (None, 2870)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 2871      
Total params: 8,151
Trainable params: 8,151
Non-trainable params: 0
_________________________________________________________________


In [8]:
m.fit(x_train, y_train,
          batch_size=10000, epochs=50, shuffle=True,
          validation_data=(x_test, y_test))

Train on 28224 samples, validate on 12096 samples
Epoch 1/50
28224/28224 [==============================] - 5s 175us/step - loss: 0.7044 - acc: 0.5365 - val_loss: 0.5848 - val_acc: 0.6836
Epoch 2/50
28224/28224 [==============================] - 1s 35us/step - loss: 0.5756 - acc: 0.7194 - val_loss: 0.5239 - val_acc: 0.7645
Epoch 3/50
28224/28224 [==============================] - 1s 35us/step - loss: 0.5040 - acc: 0.7467 - val_loss: 0.4975 - val_acc: 0.7122
Epoch 4/50
28224/28224 [==============================] - 1s 35us/step - loss: 0.4679 - acc: 0.7559 - val_loss: 0.4620 - val_acc: 0.7924
Epoch 5/50
28224/28224 [==============================] - 1s 35us/step - loss: 0.4460 - acc: 0.7957 - val_loss: 0.4294 - val_acc: 0.7908
Epoch 6/50
28224/28224 [==============================] - 1s 35us/step - loss: 0.4244 - acc: 0.7844 - val_loss: 0.4127 - val_acc: 0.8039
Epoch 7/50
28224/28224 [==============================] - 1s 36us/step - loss: 0.4087 - acc: 0.8089 - val_loss: 0.4031 - val_ac

In [31]:
_, wf, ch, _ = m.input_shape

prefix = f'triage-{wf}wf{ch}ch-'
name_model, name_metadata = make_filename(prefix=prefix, extension=('h5', 'yaml'))
print(name_model)
print(name_metadata)

triage-41wf7ch-14-Aug-2018@22-50-06.h5
triage-41wf7ch-14-Aug-2018@22-50-06.yaml


In [32]:
path_to_model = Path(path_to_models, name_model)
path_to_metadata = Path(path_to_models, name_metadata)

m.save(path_to_model)

metadata = {'path_to_x': str(path_to_x), 'path_to_y': str(path_to_x), 'input_shape': list(m.input_shape)}

with open(path_to_metadata, 'w') as f:
    yaml.dump(metadata, f)